In [1]:
import numpy as np
import pandas as pd
from nltk import * #bigrams, trigrams
import json
from collections import Counter
import ast

In [2]:
t1 = pd.read_csv("scraped_tweets1.csv")
t2 = pd.read_csv("scraped_tweets2.csv")
t3 = pd.read_csv("scraped_tweets3.csv")
t4 = pd.read_csv("scraped_tweets4.csv")

frames = [t1, t2, t3, t4]
tweets_erinDB = pd.concat(frames)

In [3]:
d = []
with open('usernames_filtered_2.txt') as rf:
    lines = rf.readlines()
    for line in lines:
        d.append(line[:-2])

tweets_erinDB_filtered = tweets_erinDB[tweets_erinDB['query'].isin(d)]

In [4]:
# read in tweets from Nathan's DB
t5 = pd.read_csv("scraped_tweets5.csv")
frames2 = [tweets_erinDB_filtered, t5]
tweets = pd.concat(frames2)

In [5]:
# remove non-english tweets, reset indices
tweets_eng = tweets[tweets['language'].isin(['en', 'und'])]
tweets_eng = tweets_eng.reset_index(drop = True)

In [6]:
# filter users who have less than 100 tweets
tweets_eng['count'] = 1
tweet_count_grouped = tweets_eng.groupby('query')
tweet_count = tweet_count_grouped['count'].agg([np.sum])
tweet_count['keep'] = tweet_count['sum'] >= 100
tweet_keep = tweet_count[tweet_count['keep'] == True]
users_tweet_keep = list(tweet_keep.index)
tweets_eng_keep = tweets_eng[tweets_eng['query'].isin(users_tweet_keep)]

#len(tweets_eng_keep), len(tweets_eng)

In [7]:
# filter users who have 3x as many friends as followers (potential bots)
friend_count_grouped = tweets_eng_keep.groupby('query')
followers_count = friend_count_grouped['num_followers'].agg([np.mean])
friends_count = friend_count_grouped['num_friends'].agg([np.mean])
friends_count['ratio'] = friends_count['mean']/friends_count['mean']
friends_count['keep'] = friends_count['ratio'] <= 3
ratio_keep = friends_count[friends_count['keep'] == True]
users_ratio_keep = list(ratio_keep.index)
tweets_trimmed = tweets_eng_keep[tweets_eng_keep['query'].isin(users_ratio_keep)]
#len(tweets_trimmed), len(tweets_eng_keep), len(tweets_eng)
tweets_trimmed = tweets_trimmed.reset_index(drop = True)

In [8]:
# deal with NaN
tweets_trimmed['entities_hashtags'] = tweets_trimmed['entities_hashtags'].fillna(0)
tweets_trimmed['entities_urls'] = tweets_trimmed['entities_urls'].fillna(0)

In [9]:
# read in lexicon
with open('ngram_map.txt', 'r') as rf:
    ngram_map = json.loads(rf.read())

In [11]:
# read in nonzero_features.txt to refill matrix
with open('nonzero_features.txt', 'r') as rf:
    nonzeros = rf.readlines()

In [12]:
data_counter = Counter()
for line in nonzeros:
    yep = line.split(' ')
    data_counter[yep[1]] += float(yep[2][:-1])

In [26]:
len(data_counter)

4284311

In [13]:
min_sum = 0.01 * len(set(tweets_trimmed['query']))
delete_set = []
for key, value in data_counter.iteritems():
    if data_counter[key] <= min_sum:
        delete_set.append(int(key))

delete_set.append(0) # feature: ""
delete_set.append(3338145) # feature: frowny face

In [14]:
# set of features left after trimming
ngram_map_trim = dict((v,k) for k,v in ngram_map.iteritems())

for index in delete_set:
    del ngram_map_trim[index]

In [15]:
len(ngram_map_trim)

65868

In [130]:
# change indices post trimming
new_map = {}
sorted_keys = sorted(ngram_map_trim.keys())

new_count = 0
for sort in sorted_keys:
    new_map[sort] = new_count
    new_count += 1

for key in ngram_map_trim.keys():
    new_value = new_map[key]
    ngram_map_trim[new_value] = ngram_map_trim.pop(key)

ngram_map_trim = dict((v,k) for k,v in ngram_map_trim.iteritems())

In [131]:
# save set of features left after trimming
with open('ngram_map_trim.txt', 'w') as wf:
    json.dump(ngram_map_trim, wf)

In [55]:
trim_feature_num = len(data_counter) - len(delete_set)

In [51]:
data = []
for line in nonzeros:
    yep = line.split(' ')
    sub_data = (yep[0], yep[1], yep[2][:-1])
    #print sub_data
    data.append(sub_data)

In [52]:
transposed = np.zeros((len(data_counter)+1, len((set(tweets_trimmed['query'])))))

In [53]:
for dat in data:
    transposed[int(dat[1])][int(dat[0])] += float(dat[2])

In [54]:
transposed.shape

(4284312, 3578)

In [132]:
trimmed_transposed = np.delete(transposed, delete_set, 0)

In [133]:
trimmed_transposed.shape

(65869, 3578)

In [134]:
user_feature_matrix = trimmed_transposed.T

In [135]:
np.savez_compressed("outputMatrix.txt", user_feature_matrix=user_feature_matrix)

In [136]:
unigrams = {}
bigrams = {}
trigrams = {}
hashtags = {}
categories = {}

#len(ast.literal_eval("never"))

for key in ngram_map_trim.keys():
    if key.startswith('#'):
        hashtags[key] = ngram_map_trim[key]
    elif '(' in key:
        if type(ast.literal_eval(key)) == tuple:
            if len(ast.literal_eval(key)) == 2:
                bigrams[key] = ngram_map_trim[key]
            else:
                trigrams[key] = ngram_map_trim[key]
    else:
        if key.isupper() or key == 'function_words' or key == 'content_words':
            categories[key] = ngram_map_trim[key]
        else:
            unigrams[key] = ngram_map_trim[key]

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [142]:
unigrams_delete = bigrams.values() + hashtags.values() + categories.values()
trimmed_transposed_unigrams = np.delete(trimmed_transposed, unigrams_delete, 0)
feature_matrix_unigrams = trimmed_transposed_unigrams.T
np.savez_compressed("outputMatrix_unigrams", matrix=feature_matrix_unigrams)

In [143]:
bigrams_delete = unigrams.values() + hashtags.values() + categories.values()
trimmed_transposed_bigrams = np.delete(trimmed_transposed, bigrams_delete, 0)
feature_matrix_bigrams = trimmed_transposed_bigrams.T
np.savez_compressed("outputMatrix_bigrams", matrix=feature_matrix_bigrams)

In [144]:
hashtags_delete = unigrams.values() + bigrams.values() + categories.values()
trimmed_transposed_hashtags = np.delete(trimmed_transposed, hashtags_delete, 0)
feature_matrix_hashtags = trimmed_transposed_hashtags.T
np.savez_compressed("outputMatrix_hashtags", matrix=feature_matrix_hashtags)

In [145]:
categories_delete = unigrams.values() + bigrams.values() + hashtags.values()
trimmed_transposed_categories = np.delete(trimmed_transposed, categories_delete, 0)
feature_matrix_categories = trimmed_transposed_categories.T
np.savez_compressed("outputMatrix_categories", matrix=feature_matrix_categories)